# Compare original median filter with normalized median filter.
Once a velocity field has been obtained from a PIV processor, there might be vectors that, visually, look as outliers. One way to get rid of them is to apply a median filter: compare every velocity vector to the average vector in its, say, 3x3 neighborhoud and, if it is bigger than a threshold, replace it with the average vector.

As described in Adrain & Westerweel, "Particle image velocimetry", 2011, J. Westerweel and coauthors created two versions of the median filter. The original one was created in 1994. Its main disadvantage is that it uses one (global) threshold for all the vectors in the velocity field. In 2005, it was proposed to normalize every vector in the velocity field before comparing it to the global threshold, thereby mitigating the disadvantage of the 1994 version of the median filter (see J. Westerweel, F. Scarano, "Universal outlier detection for PIV data", Experiments in fluids, 39(6), p.1096-1100, 2005).

OpenPIV has implemented both version of the filter. The 1994 version (the original version) is given by the function `validation.local_median_val()`. The 2005 version (the normalized version) is given by the function `validation.local_norm_median_val()`.

The phylosophy of their usage is the following. Both filters just check every vector in the velocity field and create a "mask" of the velocity field where those vector that didn't pass the threshold requirement are marked as NaNs. Then the OpenPIV function `filters.replace_outliers()` must be implemented. That function reads the "mask" and replace every NaN vector with the average of its neighbourhood.

The purpose of this tutorial is compare the two fitlers for a rather difficult case of PIV of a two-phase cap_bubbly air-water flow

In [ ]:
import pathlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
from openpiv import tools, pyprocess, preprocess, validation, filters